In [156]:
import yt_dlp
import os
import sys
import time

class ProgressHook:
    def __init__(self, update_interval=2.0):
        self.last_update_time = time.time()
        self.last_percent = None
        self.update_interval = update_interval

    def __call__(self, d):
        """
        Display download progress and handle progress bar updates.
        """
        current_time = time.time()
        if d['status'] == 'downloading':
            percent = d.get('downloaded_bytes', 0) / d.get('total_bytes', 1) * 100
            if (current_time - self.last_update_time) >= self.update_interval:
                if percent != self.last_percent:
                    sys.stdout.write(f"\r🔄 Download progress: {percent:.2f}%")
                    sys.stdout.flush()
                    self.last_percent = percent
                    self.last_update_time = current_time
        
        if d['status'] == 'finished':
            sys.stdout.write("\r\n✅ Download completed successfully!      \n")
            sys.stdout.flush()

def create_downloader(download_path):
    """Create and configure yt-dlp downloader instance."""
    options = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': os.path.join(download_path, '%(title)s.%(ext)s'),
        'ffmpeg_location': '/opt/homebrew/bin/ffmpeg',  # Modify based on your FFmpeg installation
        'progress_hooks': [ProgressHook(update_interval=2.0)],
        'quiet': True,
        'no_warnings': True
    }
    return yt_dlp.YoutubeDL(options)

def sanitize_filename(filename):
    """Sanitize filenames by removing invalid characters."""
    return ''.join(c for c in os.path.splitext(filename)[0] if c.isalnum() or c.isspace())

def process_playlist(info_dict, downloader, download_path):
    """Download all videos in a playlist."""
    playlist_dir = os.path.join(download_path, info_dict['title'])
    os.makedirs(playlist_dir, exist_ok=True)
    print(f"\n📋 Detected a playlist. Downloading playlist: '{info_dict['title']}' to '{playlist_dir}'")

    downloader.params['outtmpl']['default'] = os.path.join(playlist_dir, '%(title)s.%(ext)s')
    existing_files = set(sanitize_filename(f) for _, _, files in os.walk(playlist_dir) for f in files)

    for index, entry in enumerate(info_dict['entries']):
        video_url = "https://www.youtube.com/watch?v=" + entry['id']
        file_name = entry['title']
        sanitized_name = sanitize_filename(file_name)
        if sanitized_name not in existing_files:
            print(f"\n📥 {index + 1}. Downloading video: {file_name}")
            downloader.download([video_url])
        else:
            print(f"\n📂 {index + 1}. '{file_name}' already exists. Skipping download.")

def process_video(info_dict, url, downloader, download_path):
    """Download a single video."""
    video_file_name = sanitize_filename(info_dict.get('title', 'Unknown Title'))
    existing_files = set(sanitize_filename(f) for _, _, files in os.walk(download_path) for f in files)

    if video_file_name not in existing_files:
        print(f"\n📥 Downloading video: {info_dict.get('title', 'Unknown Title')}")
        downloader.download([url])
    else:
        print(f"\n📂 Video: {info_dict.get('title', 'Unknown Title')}' already exists. Skipping download.")

def download_video(url, download_path):
    """Download a video or playlist from YouTube."""
    downloader = create_downloader(download_path)
    
    try:
        info_dict = downloader.extract_info(url, download=False)
        if 'entries' in info_dict:
            process_playlist(info_dict, downloader, download_path)
        else:
            process_video(info_dict, url, downloader, download_path)
    except Exception as e:
        print(f"❌ An error occurred: {e}")

def main():
    """Main function to execute the video downloader."""
    print(f"""{'='*50}\n🎥 YouTube Video Downloader App - By Abhijit🧑‍💻\n{'='*50}""")
    video_url = input("Enter the YouTube video URL: ")
    download_location = input("Enter the download directory (e.g., /Users/abhijit/Downloads/): ") or '~/Downloads'

    print(f"📥 Downloading from URL: {video_url}")
    print(f"📂 Save location: {download_location}")
    
    download_location = os.path.expanduser(download_location)
    
    if os.path.isdir(download_location):
        download_video(video_url, download_location)
        print("\n🙏 Thanks for Visiting! 😊")
    else:
        print(f"❌ Error: The directory '{download_location}' does not exist.")

if __name__ == "__main__":
    main()


🎥 YouTube Video Downloader App - By Abhijit🧑‍💻
📥 Downloading from URL: https://www.youtube.com/playlist?list=PL1T8fO7ArWleyIqOy37OVXsP4hFXymdOZ
📂 Save location: /Users/abhijit/Downloads/Courses

📋 Detected a playlist. Downloading playlist: 'LLM Bootcamp - Spring 2023' to '/Users/abhijit/Downloads/Courses/LLM Bootcamp - Spring 2023'

📂 1. 'Launch an LLM App in One Hour (LLM Bootcamp)' already exists. Skipping download.

📂 2. 'LLM Foundations (LLM Bootcamp)' already exists. Skipping download.

📂 3. 'Learn to Spell: Prompt Engineering (LLM Bootcamp)' already exists. Skipping download.

📂 4. 'Augmented Language Models (LLM Bootcamp)' already exists. Skipping download.

📂 5. 'Project Walkthrough: askFSDL (LLM Bootcamp)' already exists. Skipping download.

📂 6. 'UX for Language User Interfaces (LLM Bootcamp)' already exists. Skipping download.

📂 7. 'LLMOps (LLM Bootcamp)' already exists. Skipping download.

📂 8. 'What's Next? (LLM Bootcamp)' already exists. Skipping download.

📂 9. 'Peter W